In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
import jieba
import re

def processing(text):
    # 数据清洗部分
    text = re.sub("\{%.+?%\}", " ", text)           # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("@.+?( |$)", " ", text)           # 去除 @xxx (用户名)
    text = re.sub("\u200b", " ", text)              # '\u200b'是这个数据集中的一个bad case, 不用特别在意
    # 分词
    words = [w for w in jieba.lcut(text) if w.isalpha()]
    # 对否定词`不`做特殊处理: 与其后面的词进行拼接
    while "不" in words:
        index = words.index("不")
        if index == len(words) - 1:
            break
        words[index: index+2] = ["".join(words[index: index+2])]  # 列表切片赋值的酷炫写法
    # 用空格拼接成字符串
    result = " ".join(words)
    return cc.convert(result)

In [1]:
# !pip install opencc
# !pip install regex
# !pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 778 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
stopwords_path = "/content/drive/MyDrive/文字探勘/stopword.txt"
data_set_path = "/content/drive/MyDrive/文字探勘/data_set.csv"

In [3]:
from opencc import OpenCC

cc = OpenCC('s2twp')

In [5]:
stopwords = []
with open(stopwords_path, "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(cc.convert(w.strip()))

In [6]:
df_train = pd.read_csv(data_set_path)

In [7]:
df_train = df_train.dropna()

In [8]:
len(df_train)

82522

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern='\[?\w+\]?', 
                             stop_words=stopwords)
X_train = vectorizer.fit_transform(df_train["text"])
y_train = df_train["label"]

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['元', '噸', '數', '末'] not in stop_words.
  warnings.warn(


In [10]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

x_train_train, x_train_evaluate, y_train_train, y_train_evaluate = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

In [11]:
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

SVM_model = SVC(C=1.0, kernel='linear', degree=3, gamma='auto', random_state=1)
SVM_model.fit(x_train_train, y_train_train)

SVC(gamma='auto', kernel='linear')

## Evalutaion


In [12]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.preprocessing import label_binarize

In [13]:
y_train_evaluate_test = SVM_model.predict(x_train_evaluate)

print(metrics.classification_report(y_train_evaluate, y_train_evaluate_test, labels=[i for i in range(0,2)]))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      4185
           1       0.92      0.90      0.91      4068

    accuracy                           0.91      8253
   macro avg       0.91      0.91      0.91      8253
weighted avg       0.91      0.91      0.91      8253



## Test

In [14]:
strs = ["""
電信 產業今年最重要的一件事就是「合併」，繼去年台灣大宣布併購台灣之星後，今年遠傳也提出併購亞太電（ 3682-TW ）信，將對兩大電信併購案進行討論，而併購案的後續發展將攸關電信業的未來，投資人緊盯 電信三雄 之間的角力。

電信三雄11月、前11月成績單出爐，中華電(2412)在單月、累計獲利均穩居雙冠王，進一步就成長率表現來看，11月及前11月EPS年增率遠傳(4904)均有雙位數表現 中華電居次、台灣大(3045)第三。另外，以本業行動服務營收來看，11月三家行動服務營收都呈現正成長，以中華電年增5%最高，其次是遠傳、台灣大。

電信三雄11月單月、累計財報出爐，就實際數字來說，中華電(2412)11月及前11月每股盈餘(EPS)分別為0.36元及4.35元穩居雙冠王；台灣大(3045)位居第二，11月及前11月EPS分別為0.31元及3.64元；第三名則為遠傳(4904)，11月及前11月EPS分別為0.25元及2.67元。惟就成長率來看，遠傳穩居獲利成長冠軍，11月EPS年增26.1%，連續11個月年成長率達雙位數，且以11月稅後淨利來看，與去年同期相比，遠傳EPS年增率達26.1%，表現第一，中華電年增6.1%居次，台灣大則是年增3.6%；以累計前11月稅後淨利來看，遠傳年增率20.4%表現居冠(扣除去年7月活化資產的營業外收益影響)，其次是中華電年增3.1%居次，台灣大則年增1.2%。

比較電信三雄EBITDA(稅息折攤前獲利)，11月台灣大EBITDA年增率7.5%表現最優，其次為遠傳年增7.1%居次，中華電則是年增3.8%。若以前11月EBITDA累計數來看，以遠傳年增9.1%表現居冠，其次是中華電信年增4.3%，台灣大則年增3.8%。

回歸電信三雄本業行動服務營收來看，11月三家行動服務營收都呈現正成長，都已經連續20~21個月為正成長。中華電11月行動服務營收年增率5%、位居第一，其次為遠傳的年增3.6%，台灣大年增率則為2.4%；值得一提的是，遠傳11月行動服務營收達42.46億元，從2019年8月以來屢創新高，而遠傳與台灣大單月差距為1.4億、再創新高，更是已經連續37個月超越台灣大。", "烦死了！为什么周末还要加班[愤怒]
""",
"""
3Q22 營運表現，營收 404.68 億元 (QoQ -2.13%、YoY +8.23%)，稅後淨利 28.39 億元(QoQ -5.17%、YoY +0.06%)，毛利率 19.69%，營益率 10.08%，淨利率 7.02%，EPS 1.0 元。
2020-2021 年在疫情趨動台灣電商滲透率有加速提升，Euromonitor 預估 2022 年台灣電商滲透率約 17%，相較於南韓的 45%和中國的 31%，顯現市場長線成長空間仍大，MOMO 今年前 11 月營收 YoY+16.8%恐難達成先前設定之成長兩成目標，至於在零售業務營益率表現上，因追求營收高成長為首要目標，2H22 持續擴大行銷資源投放並給予消費者更好產品價格，加計去年疫情助攻致使營益率基期較高，2022 年營益率表現將不如去年，此對台灣大獲利挹注成長性降低，此外，公司對於衛星倉建設態度不如過去積極，目標轉為提升效率為主，物流則為近年資本支出重點，南區和中區物流中心預計分別於 4Q23 和 2025 年加入營運，並同步擴大自營車隊規模，預計 2022 年自有物流占比達 25%。
我們對於 2022 年的預期，預估營收 1,725.32 億元 (YoY +10.52%)，稅後淨利 114.13 億元，EPS 3.24 元，目標價由 103.0 元下調至 101.0 元，投資評等維持中立。
""",
"""
前一波明星汽車2201，在11/29爆天量後

走震盪整理，從型態上，難辨多空

就是無數醜k堆疊

過程中，正動能持續收斂

但同族群東陽/三陽，破線

開始讓人憂心

資金的利用效率重要

步伐穩，但不等於站在原地挨打

這一波能做的中小生技強股就那幾檔

盤爛少做，合邏輯

若要做，只能做最強的逆勢股

但如12/16直播所說

盤爛出門補魚，盤中的鎖單策略重要

目前強股為南光/松瑞藥/強生/濟生/聯合

已連續第三天出現在小P程式選股內

少數已出現高位風險燈，小P已在選出第二天進場

若你此刻才看到，都慢了好多拍囉

就不建議亂追

可靜等另一批強股出現再做

一樣的邏輯

盤爛，看到強股，多半不做能省波動風險

要做，手腳要快，且記得切入的手段
""",
"""
由於2040台灣年要達到禁售燃油車，因此政府將會更快速的推動電動車的普及化，然而目前的電動車價格較高，而電動機車也是，此時政府勢必會補助電動車，而這麼做的情況下來，對gogoro來說將有助於將電動機車市佔率的穩定，以及增加銷售量及利潤。
對投資人而言，要看的是一家企業的未來，投資gogoro未來不會賠錢，反而有機會致富。
從gogoro的經營來看，目前充電站數量比加油站還多，且是佔率高達80%以上，只是目前售價較高，所以很依賴政府補助，才能夠讓銷售量增加。
""",
"""
美東時間周五，美股三大指數震蕩整理，截至發稿，道指跌0.13%，納指跌0.14%，標普500指數跌0.07%。中概新能源汽車股普跌，理想汽車跌超8%，Q3凈虧損從上年同期的2151萬元擴大至16.4億元且遜於預期。摩根大通表示，對公司長期盈利前景逐漸樂觀，目標價升至30美元，維持增持評級。愛立信漲超4%，消息稱公司與蘋果簽署全球專利許可協議。

盤前公布的數據显示，美國11月PPI同比上漲7.4%，預估為7.2%，前值為8.0%；美國11月PPI環比上漲0.3%，預估為0.2%，前值為0.2%。

美聯儲將於下周三公布利率決議，周五的PPI和下周二的CPI預計將在未來幾個月對利率走勢產生重大影響。

市場普遍預計12月將加息50個基點，但對終端利率的預測，以及對利率將如何影響經濟的擔憂導致美股最近出現震蕩，並中斷了始於10月的反彈勢頭。

Hargreaves Lansdown高級投資和市場分析師Susannah Streeter表示：“目前市場對此非常敏感，儘管超大規模的加息可能已經成為過去，但問題是漸進式加息將持續多久，這就是為什麼你會看到這兩個惡魔：衰退和高通脹。真正的擔憂是，我們將面臨滯漲。”
""",
"""
大家晚上好

台股今日開低走低，盤中一度要跌破14400點，離半年線14300僅隔一條街的距離😱😱😱

今日最耀眼的族群當屬生技族群。尤其以前陣子藥荒題材為主軸，蔓延至其它個股。

口罩、額溫槍、血氧機等防疫概念股，在中國全面解封題材下，也受到資金的青睞！

基本上中國正在走台灣當初走過的路。相關個股舊瓶新裝，繼續拿出來炒。

不過這邊大多只是反彈，上方無數冤魂等著下一批人來抓交替☠️☠️☠️

厲害的人，搶搶短線可以，但記得出場也要即時出場！

個人目前的主軸放在藥荒題材上，週末專欄大篇幅介紹以及可佈局個股，相信有看過的同學對於目前走勢都了然於心！

也不少同學今日有把握到這一波😍😍😍

整體盤勢除了生技族群外，多數族群大多遭逢不小跌幅，僅剩個股表現。
""",
"""
2,我持續看多大盤,看多的邏輯是資金面對台股有利,股市的漲跌主要都是資金面決定的.美國通膨在今年6月見頂,最高的通膨是CPI年增9.1%.目前CPI的數字已經回跌到了7.1%,而隨著未來幾個月基期越來越高,CPI數字下滑的速度會越來越快,CPI已經連續兩個月的下滑速度優於預期.這幾天的下跌是在跌鮑爾的鷹派發言,我總說因為鮑爾的發言漲跌都沒意義,鮑爾會唱鷹調很正常,因為他最重要的工作就是控制通膨,股市過快上漲對消費是有提振的,他目前也不希望股市漲太多,但其實道瓊已經重回多頭走勢了.我也說過如果因為鷹派發言而下跌,會是好的買點,我是預判如果是鴿派發言大盤也漲不上去了,因為大盤漲了2300點本來就需要回檔,所以大盤就會橫盤以盤代跌,因為大盤還不到急攻的條件,需要用時間去取待空間的回檔,但如果我們預判2023年是大多頭,正好可以趁這波回檔勇敢加碼迎接2023年的主升段.
""",
"""
美元指數週五小漲0.14收104.84,美股道瓊下跌0.85%,那斯達克下跌0.97%,費半下跌0.95%,TSM下跌1.69%.TSM目前折價0.25%,這會對今天台積電的走勢有所壓抑,長期TSM有3%的溢價,要扭轉這樣的溢價很難.但如果台積電今天的跌幅不超過3%也會帶動TSM走強.週五的台指期夜盤是下跌94點,這也預告今天台股有向下修正的壓力,技術派的人會說大盤如果跌破11月22日的低點14371的支撐就會走空,因為月線就會反轉向下走空,但我早就說大盤漲了2300點需要有回檔,但這回檔看起來會在時間上拖比較久,但幅度不會太大,我覺得14000點不會破,也就是大盤最多只有500點的下跌空間.
""",
"""
前天股價大跌排名第一的Moderna 昨晚大漲19.63%,原因是他的mRNA癌症疫苗跟默克藥廠的免疫藥物Keytruda組合療法,能將晚期黑色素癌的死亡率降低44%.2030年前mRNA疫苗普遍用於癌症的預防跟治療(復發的預防)的可能性大增.昨天BNT也上漲7.97%,隨著德國批准科興疫苗可以給在德國的中國人施打,那麼中國有可能也批准在中國的外國人可以施打BNT疫苗.
""",
"""
昨天美股漲幅不如預期,是因為人們還是擔心FED不知道會怎麼說,但真實的狀況是通膨本來就是人造危機,是FED過去無限QE的產物,而且FED太慢採取行動升息並中止這荒謬的無限QE,當FED採取行動後通膨問題就不該存在了,其實FED三月才開始升息,這個升息循環正好在俄烏戰爭開始後開始,讓2022年全球的經濟一片哀嚎.但事實是通膨年增6月就見頂了,也就是FED開始升息縮表後三個月通膨年增就見頂了,下個月CPI數字就會開始月減,但FED為了管理人們的預期,不斷的放鷹恐嚇投資人.這個恐嚇跟中國官方過去恐嚇他們的人民Omicron有多可怕一樣,最後謊言都會不攻自破.他們都認為他們說謊是為了人民的生命財產安全,但謊言只能短時間奏效,最後謊言都會變成笑話.我們現在還在擔心FED謊言的威力?我覺得他們不敢再繼續說謊下去了,一個月一個月CPI的下滑速度會越來越快,我們該擔心的是通縮而不是通膨,科技的發展讓物價長時間是很明顯的呈現下降趨勢,就短短一天癌症疫苗也有很大的進展,無限能源也有很大的進展,股市應該要大漲慶祝才對啊!
""",
"""
美股道瓊上漲0.55%,那斯達克上漲1.13%,費半更是大漲2.67%,半導體股漲幅第一的是Nvidia,大漲了6.51%,取代上漲1.97%的TSM成為美股市值排名第十位的股票,Nvidia的大漲說明半導體的夢又來了,Nvidia代表高速運算,代表數據中心,代表AI,代表元宇宙,NvidiaQ3是衰退的,而且一直到明年的上半年都不會好,但目前Nvidia還是有70倍的本益比,這說明了半導體產業特別是高速運算長期還是被看的非常看好的.台積電到美國設廠,對台積電是好是壞不好說,但對蘋果,Nvidia等公司的晶片生產安全是有保障的.
""",
"""
昨天大盤第指數三天下跌,下跌0.53%,但櫃買卻上漲0.31%,也就是外資賣超90.5億,只讓台股下跌77點,融資餘額增加9.7億,顯然投資人信心有點恢復了.但融資餘額在這波反彈2500點(20%)的上漲波段中,只從1653億增加到了1687億,只增加了2%.也就是底部都不是融資戶買的,所以這個位階是很安全的,因為融資戶90%都是虧錢的,跟他們對做會有很大的勝率.當然如果大盤能多回檔一點,價值投資者的安全邊際就很大.我長期很看好健策,也在這檔股票上賺不少錢,但最近半年我都沒有持股.最大的原因就是健策的投信持股比例始終在20%以上,但即便如此只要安全邊際夠大,我也會買.這波我就設定370以下買,前天最低379,昨天大盤跌本來覺得有機會買回我的最愛,但昨天健策又逆勢上漲到397.5元,只能繼續等.其實我對每檔股票都會有一套公式去算這檔股票的內在價值,一般在內在價值高於股價20%的時候我才會出手買,但370的健策也只有10%的安全邊際,會考慮買也是因為偏好.而370的價位也只會買進可用資金的2%,會在更低的價格更大部位的買進.所以大盤連跌三天我心情都很好,我期實期待能多跌一點(準確的說是多波動一點),因為有很多想買還沒買到的股票可以買.但最後根本都沒有跌到我想要的價位,昨晚美股漲,這些價位今天是更不會來了.
"""
]
words = [processing(s) for s in strs]
vec = vectorizer.transform(words)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.845 seconds.
DEBUG:jieba:Loading model cost 0.845 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [15]:
output = SVM_model.predict(vec)
output

array([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1])